# Continuum Data
How does the continuum data fit into our data parsing/storage model that we've implemented thus far?

In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
import h5py
import fiasco
import plasmapy
import astropy.units as u
import ChiantiPy.tools.io as ch_io
os.environ['XUVTOP'] = '/Users/willbarnes/ssw/packages/chianti/dbase'

%matplotlib inline

 using cli
 using CLI for selections


The relevant continuum files that we need to read are:
* ~~`gffgu.dat`~~ -- no ion/element specific information (Sutherland)
* ~~`gffint.dat`~~ -- no ion/element specific information (Sutherland)
* ~~`heseq_2photon.dat`~~ -- element specific information, can be split across multiple element datasets
* ~~`hseq_2photon.dat`~~ -- element specific information, can be split across multiple element datasets
* ~~`itoh.dat`~~ -- element specific information, can be split across multiple element datasets
* ~~`klgfb.dat`~~ -- no ion/element specific information (only principal quantum number and angular momentum)
* ~~`verner_short.txt`~~ -- ion and element specific information, can be split across multiple ion datasets


Need to implement the following three methods for continuum calculations
* `free_free_continuum` (i.e. bremsstrahlung)
* `free_bound_continuum`
* `two_photon_continuum`

Then there's also the issue of radiative losses associated with each of these processes. 

## Parser Prototypes

In [ ]:
class GffguParser(fiasco.io.GenericParser):
    filetype = 'gffgu'
    dtypes = [float, float, float]
    units = [u.dimensionless_unscaled, u.dimensionless_unscaled, u.dimensionless_unscaled]
    headings = ['u', 'gamma_squared', 'gaunt_factor']
    descriptions = ['scaled frequency','scaled temperature','free-free Gaunt factor']
    
    def __init__(self, filename, **kwargs):
        super().__init__(filename, **kwargs)
        self.full_path = os.path.join(self.ascii_dbase_root, 'continuum', filename)
        self.body_index = 5
    
    def preprocessor(self, table, line, index):
        if index >= self.body_index and '--' not in line:
            super().preprocessor(table, line, index)
    
    def extract_footer(self, lines):
        comment = []
        for i, l in enumerate(lines):
            if i <= self.body_index - 3:
                comment.append(l)
            else:
                break
        
        footer = '\n'.join([l.strip() for l in comment ])
        return footer
    
    def to_hdf5(self, hf, df, **kwargs):
        grp_name = '/'.join(['continuum', self.filetype])
        if grp_name not in hf:
            grp = hf.create_group(grp_name)
            grp.attrs['chianti_version'] = df.meta['chianti_version']
            grp.attrs['footer'] = df.meta['footer']
        else:
            grp = hf[grp_name]
            
        for name in df.colnames:
            col = df[name]
            if type(col) == u.Quantity:
                data = col.value
            else:
                data = col.data
            if '<U' in data.dtype.str:
                numchar = data.dtype.str[2:]
                data = data.astype('|S{}'.format(numchar))
            if name in grp:
                ds = grp[name]
            else:
                if data.dtype == np.dtype('O'):
                    ragged_dtype = h5py.special_dtype(vlen=np.dtype('float64'))
                    ds = grp.create_dataset(name, data=data, dtype=ragged_dtype)
                else:
                    ds = grp.create_dataset(name, data=data, dtype=data.dtype)
            if col.unit is None:
                ds.attrs['unit'] = 'SKIP'
            else:
                ds.attrs['unit'] = col.unit.to_string()
            ds.attrs['description'] = df.meta['descriptions'][name]
            

In [ ]:
class GffintParser(GffguParser):
    filetype = 'gffint'
    dtypes = [float, float, float, float, float]
    units = [u.dimensionless_unscaled, u.dimensionless_unscaled, u.dimensionless_unscaled, u.dimensionless_unscaled, u.dimensionless_unscaled]
    headings = ['log_gamma_squared', 'gaunt_factor', 's1', 's2', 's3']
    descriptions = ['log scaled temperature', 'total free-free Gaunt factor', 'spline coefficient',
                    'spline coefficient', 'spline coefficient']
    
    def __init__(self, filename, **kwargs):
        super().__init__(filename, **kwargs)
        self.body_index = 4
        

In [ ]:
class KlgfbParser(fiasco.io.GenericParser):
    filetype = 'klgfb'
    dtypes = [int, int, float, float]
    units = [None, None, u.dimensionless_unscaled, u.dimensionless_unscaled]
    headings = ['n', 'l', 'log_pe', 'log_gaunt_factor']
    descriptions = ['principal quantum number', 'orbital angular momentum number', 
                    'log photon energy divided by ionization potential', 'log free-bound Gaunt factor']
    
    def __init__(self, filename, **kwargs):
        super().__init__(filename, **kwargs)
        self.full_path = os.path.join(self.ascii_dbase_root, 'continuum', filename)
        
    def preprocessor(self, table, line, index):
        if index == 0:
            pass
        elif index == 1:
            self._photon_energy = np.array(line.strip().split(), dtype=float)
        else:
            line = line.strip().split()
            gf = np.array(line[2:], dtype=float)
            table.append(line[:2] + [self._photon_energy, gf])
            
    def extract_footer(self, *args):
        return """Log of free-bound Gaunt factors as a function of log of photon energy divided by ionization potential
From Karzas, W. J. and Latter, R., 1961, ApJS, 6, 167
"""
    
    def to_hdf5(self, hf, df, **kwargs):
        grp_name = '/'.join(['continuum', self.filetype])
        if grp_name not in hf:
            grp = hf.create_group(grp_name)
            grp.attrs['chianti_version'] = df.meta['chianti_version']
            grp.attrs['footer'] = df.meta['footer']
        else:
            grp = hf[grp_name]
            
        for name in df.colnames:
            col = df[name]
            if type(col) == u.Quantity:
                data = col.value
            else:
                data = col.data
            if '<U' in data.dtype.str:
                numchar = data.dtype.str[2:]
                data = data.astype('|S{}'.format(numchar))
            if name in grp:
                ds = grp[name]
            else:
                if data.dtype == np.dtype('O'):
                    ragged_dtype = h5py.special_dtype(vlen=np.dtype('float64'))
                    ds = grp.create_dataset(name, data=data, dtype=ragged_dtype)
                else:
                    ds = grp.create_dataset(name, data=data, dtype=data.dtype)
            if col.unit is None:
                ds.attrs['unit'] = 'SKIP'
            else:
                ds.attrs['unit'] = col.unit.to_string()
            ds.attrs['description'] = df.meta['descriptions'][name]
            

In [ ]:
class VernerParser(fiasco.io.GenericParser):
    filetype = 'verner_short'
    dtypes = [int, int, int, int, float, float, float, float, float, float]
    units = [None, None, None, None, u.eV, u.eV, u.megabarn, u.dimensionless_unscaled, u.dimensionless_unscaled,
             u.dimensionless_unscaled]
    headings = ['Z', 'n_electrons', 'n', 'l', 'E_thresh', 'E_0_fit', 'sigma_0', 'y_a_fit', 'P_fit', 'y_w_fit']
    descriptions = ['atomic number', 'number of electrons', 'principal quantum number',
                    'orbital angular momentum number', 'threshold energy below which cross-section is 0',
                    'E_0 fit parameter', 'nominal value of cross-section', 'y_a fit parameter', 'P fit parameter',
                    'y_w fit parameter']
    
    def __init__(self, filename, **kwargs):
        super().__init__(filename, **kwargs)
        self.full_path = os.path.join(self.ascii_dbase_root, 'continuum', filename)
        
    def extract_footer(self, *args):
        return """Fit parameters for calculating partial photoionization cross-sections for individual ions
From Verner, D. A . and Yakovlev, D. G., 1995, A&AS, 109, 125
"""
    
    def to_hdf5(self, hf, df, **kwargs):
        for row in df:
            el = plasmapy.atomic.atomic_symbol(int(row['Z'])).lower()
            stage = row['Z'] - row['n_electrons'] + 1
            grp_name = f'{el}/{el}_{stage}/continuum/{self.filetype}'
            if grp_name not in hf:
                grp = hf.create_group(grp_name)
                grp.attrs['chianti_version'] = df.meta['chianti_version']
                grp.attrs['footer'] = df.meta['footer']
            else:
                grp = hf[grp_name]
            for col in row.colnames:
                if col == 'Z' or col == 'n_electrons':
                    continue
                ds = grp.create_dataset(col, data=row[col])
                ds.attrs['description'] = df.meta['descriptions'][col]
                if not hasattr(row[col], 'unit'):
                    ds.attrs['unit'] = 'SKIP'
                else:
                    ds.attrs['unit'] = row[col].unit.to_string()

In [ ]:
class ItohParser(fiasco.io.GenericParser):
    filetype = 'itoh'
    dtypes = [int, float]
    units = [None, u.dimensionless_unscaled]
    headings = ['Z', 'a']
    descriptions = ['atomic number', 'fit coefficient']
    
    def __init__(self, filename, **kwargs):
        super().__init__(filename, **kwargs)
        self.full_path = os.path.join(self.ascii_dbase_root, 'continuum', filename)
        
    def preprocessor(self, table, line, index):
        a_matrix = np.array(line.strip().split()).reshape((11,11))
        table.append([index+1] + [a_matrix])
    
    def extract_footer(self, *args):
        return """Analytic fit coefficients as a function of scaled temperature and energy for calculating the relativistic free-free Gaunt factor
From Itoh, N., et al., ApJS, 2000, 128, 125
"""
    
    def to_hdf5(self, hf, df, **kwargs):
        for row in df:
            el = plasmapy.atomic.atomic_symbol(int(row['Z'])).lower()
            grp_name = f'{el}/continuum/{self.filetype}'
            if grp_name not in hf:
                grp = hf.create_group(grp_name)
                grp.attrs['chianti_version'] = df.meta['chianti_version']
                grp.attrs['footer'] = df.meta['footer']
            else:
                grp = hf[grp_name]
            for col in row.colnames:
                if col == 'Z':
                    continue
                ds = grp.create_dataset(col, data=row[col])
                ds.attrs['description'] = df.meta['descriptions'][col]
                if not hasattr(row[col], 'unit'):
                    ds.attrs['unit'] = 'SKIP'
                else:
                    ds.attrs['unit'] = row[col].unit.to_string()

In [ ]:
class HSeqParser(fiasco.io.GenericParser):
    filetype = 'hseq_2photon'
    dtypes = [int, float, int, float, float, float]
    units = [None, u.dimensionless_unscaled, None, 1/u.s, 1/u.s, u.dimensionless_unscaled]
    headings = ['Z', 'y', 'Z_0', 'A', 'A_sum', 'psi']
    descriptions = ['atomic number', 'fraction of energy carried by one of the two photons', 'nominal atomic number',
                    'radiative decay rate', 'summed radiative decay rate', 'spectral distribution function']
    
    def __init__(self, filename, **kwargs):
        super().__init__(filename, **kwargs)
        self.full_path = os.path.join(self.ascii_dbase_root, 'continuum', filename)
        
    def preprocessor(self, table, line, index):
        if index == 0:
            self._y0 = np.array(line.strip().split(), dtype=float)
        elif index == 1:
            self._z0 = np.array(line.strip().split(), dtype=float)
        else:
            line = line.strip().split()
            table.append([line[0]] + [self._y0] + [self._z0] + line[1:3] + [np.array(line[3:])])
    
    def extract_footer(self, *args):
        return """Information needed for calculating two-photon continuum emission for hydrogen isoelectronic sequence
Radiative decay rates from Parpia, F. A., and Johnson, W. R., 1982, Phys. Rev. A, 26, 1142
Spectral distribution function from Goldman, S.P. and Drake, G.W.F., 1981, Phys Rev A, 24, 183
"""
    
    def to_hdf5(self, hf, df, **kwargs):
        for row in df:
            el = plasmapy.atomic.atomic_symbol(int(row['Z'])).lower()
            grp_name = f'{el}/continuum/{self.filetype}'
            if grp_name not in hf:
                grp = hf.create_group(grp_name)
                grp.attrs['chianti_version'] = df.meta['chianti_version']
                grp.attrs['footer'] = df.meta['footer']
            else:
                grp = hf[grp_name]
            for col in row.colnames:
                if col == 'Z':
                    continue
                ds = grp.create_dataset(col, data=row[col])
                ds.attrs['description'] = df.meta['descriptions'][col]
                if not hasattr(row[col], 'unit'):
                    ds.attrs['unit'] = 'SKIP'
                else:
                    ds.attrs['unit'] = row[col].unit.to_string()

In [ ]:
class HeSeqParser(fiasco.io.GenericParser):
    filetype = 'heseq_2photon'
    dtypes = [int, float, float, float]
    units = [None, u.dimensionless_unscaled, 1/u.s, u.dimensionless_unscaled]
    headings = ['Z', 'y', 'A', 'psi']
    descriptions = ['atomic number', 'fraction of energy carried by one of the two photons', 'radiative decay rate',
                    'spectral distribution function']
    
    def __init__(self, filename, **kwargs):
        super().__init__(filename, **kwargs)
        self.full_path = os.path.join(self.ascii_dbase_root, 'continuum', filename)
        
    def preprocessor(self, table, line, index):
        if index == 0:
            self._y0 = np.array(line.strip().split(), dtype=float)
        else:
            line = line.strip().split()
            table.append([line[0]] + [self._y0] + line[1:2] + [np.array(line[2:])])
    
    def extract_footer(self, *args):
        return """Information needed for calculating two-photon continuum emission for helium isoelectronic sequence
Radiative decay rates from Drake, G.W.F., 1986, Phys. Rev. A, 34, 2871
Spectral distribution function from Drake, G.W.F., Victor, G.A., Dalgarno, A., 1969, Phys. Rev. A, 180, 25.
"""
    
    def to_hdf5(self, hf, df, **kwargs):
        for row in df:
            el = plasmapy.atomic.atomic_symbol(int(row['Z'])).lower()
            grp_name = f'{el}/continuum/{self.filetype}'
            if grp_name not in hf:
                grp = hf.create_group(grp_name)
                grp.attrs['chianti_version'] = df.meta['chianti_version']
                grp.attrs['footer'] = df.meta['footer']
            else:
                grp = hf[grp_name]
            for col in row.colnames:
                if col == 'Z':
                    continue
                ds = grp.create_dataset(col, data=row[col])
                ds.attrs['description'] = df.meta['descriptions'][col]
                if not hasattr(row[col], 'unit'):
                    ds.attrs['unit'] = 'SKIP'
                else:
                    ds.attrs['unit'] = row[col].unit.to_string()

## Testing Prototypes

In [2]:
p = fiasco.io.Parser('gffgu.dat')
df = p.parse()
df

u,gamma_squared,gaunt_factor
,,
float64,float64,float64
0.0001,0.0001,5.524282
0.0001,0.00012589254,5.397506
0.0001,0.00015848932,5.270733
0.0001,0.00019952623,5.143968
0.0001,0.00025118864,5.017267
0.0001,0.00031622777,4.890607
0.0001,0.00039810717,4.763966
0.0001,0.00050118723,4.637384


In [3]:
df.meta

OrderedDict([('footer',
              'Free-Free Emission Gaunt factors for Maxwellian electrons\nas a function of scaled temperature and scaled frequency.\nv1.0.0, Sutherland 1997'),
             ('chianti_version', '8.0.2'),
             ('filename', 'gffgu.dat'),
             ('descriptions',
              {'gamma_squared': 'scaled temperature',
               'gaunt_factor': 'free-free Gaunt factor',
               'u': 'scaled frequency'})])

In [4]:
p = fiasco.io.Parser('gffint.dat')
df = p.parse()
df

log_gamma_squared,gaunt_factor,s1,s2,s3
,,,,
float64,float64,float64,float64,float64
-4.0,1.113883,0.01348,0.010471,-0.001854972
-3.8,1.116983,0.0174458,0.009358014,0.005564917
-3.6,1.120891,0.0218568,0.01269696,0.00497025
-3.4,1.12581,0.02753201,0.01567911,0.00642914
-3.2,1.131995,0.03457515,0.0195366,0.007563143
-3.0,1.139752,0.04329737,0.02407448,0.008818313
-2.8,1.149445,0.05398536,0.02936547,0.009538678
-2.6,1.161493,0.06687619,0.03508868,0.009776803


In [ ]:
df.meta

In [5]:
p = fiasco.io.Parser('klgfb.dat')
df = p.parse()
df

n,l,log_pe [41],log_gaunt_factor [41]
,,,
int64,int64,float64,float64
1,0,0.0 .. 18.42068,-0.22652 .. -7.27506
2,0,0.0 .. 18.42068,-0.06763 .. -5.19604
2,1,0.0 .. 18.42068,-0.15466 .. -23.61664
3,0,0.0 .. 18.42068,0.06015 .. -3.97977
3,1,0.0 .. 18.42068,0.09348 .. -21.41962
3,2,0.0 .. 18.42068,-0.27102 .. -40.24596
4,0,0.0 .. 18.42068,0.16467 .. -3.11728
4,1,0.0 .. 18.42068,0.21994 .. -19.92853


In [ ]:
df.meta

In [6]:
p = fiasco.io.Parser('verner_short.txt')
df = p.parse()
df

Z,n_electrons,n,l,E_thresh,E_0_fit,sigma_0,y_a_fit,P_fit,y_w_fit
,,,,eV,eV,Mbarn,,,
int64,int64,int64,int64,float64,float64,float64,float64,float64,float64
1,1,1,0,13.6,0.4298,54750.0,32.88,2.963,0.0
2,1,1,0,54.42,1.72,13690.0,32.88,2.963,0.0
2,2,1,0,24.59,5.996,4470.0,2.199,6.098,0.0
3,1,1,0,122.5,3.871,6083.0,32.88,2.963,0.0
3,2,1,0,75.64,20.06,320.1,7.391,2.916,0.0
3,3,2,0,5.392,3.466,47.74,20.35,4.423,0.0
4,1,1,0,217.7,6.879,3422.0,32.88,2.963,0.0
4,2,1,0,153.9,17.6,545.8,17.19,3.157,0.0


In [ ]:
df.meta

In [7]:
p = fiasco.io.Parser('itoh.dat')
df = p.parse()
df

Z,"a [11,11]"
,
int64,float64
1,2.42979 .. 10.9661
2,2.43018 .. -3.92613
3,2.40975 .. -24.8316
4,2.37989 .. -24.4755
5,2.34678 .. 16.8265
6,2.31324 .. 29.224
7,2.27996 .. 18.7566
8,2.24788 .. 6.80918


In [ ]:
df.meta

In [8]:
p  = fiasco.io.Parser('hseq_2photon.dat')
df_h = p.parse()
df_h

Z,y [17],Z_0 [4],A,A_sum,psi [17]
,,,1 / s,1 / s,
int64,float64,int64,float64,float64,float64
1,0.0 .. 1.0,1 .. 92,8.229,1.884,0.0 .. 0.0
2,0.0 .. 1.0,1 .. 92,526.6,1.884,0.0 .. 0.0
3,0.0 .. 1.0,1 .. 92,5997.0,1.884,0.0 .. 0.0
4,0.0 .. 1.0,1 .. 92,33690.0,1.883,0.0 .. 0.0
5,0.0 .. 1.0,1 .. 92,128500.0,1.882,0.0 .. 0.0
6,0.0 .. 1.0,1 .. 92,383500.0,1.881,0.0 .. 0.0
7,0.0 .. 1.0,1 .. 92,966600.0,1.88,0.0 .. 0.0
8,0.0 .. 1.0,1 .. 92,2153000.0,1.879,0.0 .. 0.0


In [9]:
p  = fiasco.io.Parser('heseq_2photon.dat')
df_he = p.parse()
df_he

Z,y [41],A,psi [41]
,,1 / s,
int64,float64,float64,float64
2,0.0 .. 1.0,50.94,0.0 .. 0.0
3,0.0 .. 1.0,1939.0,0.0 .. 0.0
4,0.0 .. 1.0,18140.0,0.0 .. 0.0
5,0.0 .. 1.0,92020.0,0.0 .. 0.0
6,0.0 .. 1.0,329600.0,0.0 .. 0.0
7,0.0 .. 1.0,953700.0,0.0 .. 0.0
8,0.0 .. 1.0,2306000.0,0.0 .. 0.0
9,0.0 .. 1.0,5021000.0,0.0 .. 0.0


## Testing HDF5 Writers

In [ ]:
%%bash
rm /Users/willbarnes/Desktop/test_continuum.h5

In [ ]:
with h5py.File('/Users/willbarnes/Desktop/test_continuum.h5','a') as hf:
    # gffgu
    p = fiasco.io.Parser('gffgu.dat', custom_parser=GffguParser)
    df = p.parse()
    p.to_hdf5(hf, df)
    # gffint
    p = fiasco.io.Parser('gffint.dat', custom_parser=GffintParser)
    df = p.parse()
    p.to_hdf5(hf, df)
    # klgfb
    p = fiasco.io.Parser('klgfb.dat', custom_parser=KlgfbParser)
    df = p.parse()
    p.to_hdf5(hf, df)
    # itoh
    p = fiasco.io.Parser('itoh.dat', custom_parser=ItohParser)
    df = p.parse()
    p.to_hdf5(hf, df)
    # 2photon
    p = fiasco.io.Parser('hseq_2photon.dat', custom_parser=HSeqParser)
    df = p.parse()
    p.to_hdf5(hf, df)
    p = fiasco.io.Parser('heseq_2photon.dat', custom_parser=HeSeqParser)
    df = p.parse()
    p.to_hdf5(hf, df)
    # verner
    p = fiasco.io.Parser('verner_short.txt', custom_parser=VernerParser)
    df = p.parse()
    p.to_hdf5(hf, df)

In [ ]:
foo = fiasco.base.DataIndexer('/Users/willbarnes/Desktop/test_continuum.h5','/')
foo

In [ ]:
foo['continuum']

In [ ]:
sorted(foo['al'].fields, key=lambda x: int(x.split('_')[-1]) if len(x.split('_')) > 1 else 0)

In [ ]:
foo['al']['continuum']

In [ ]:
foo['al']['al_1']['continuum']['verner_short'].as_table()

In [ ]:
a, b = os.path.splitext('al_1.fblvl')

In [ ]:
a

In [ ]:
foo = '/Users/willbarnes/ssw/packages/chianti/dbase/al/al_1/al_1.diparams'

In [ ]:
os.path.basename(foo) == 

## Raw Files

In [ ]:
%%bash 
head -n 50 /Users/willbarnes/ssw/packages/chianti/dbase/continuum/gffint.dat

In [ ]:
%%bash 
head -n 10 /Users/willbarnes/ssw/packages/chianti/dbase/continuum/gffgu.dat

In [ ]:
%%bash 
tail -n 4 /Users/willbarnes/ssw/packages/chianti/dbase/continuum/klgfb.dat

In [ ]:
%%bash 
head -n 10 /Users/willbarnes/ssw/packages/chianti/dbase/continuum/verner_short.txt

In [ ]:
%%bash 
tail -n 1 /Users/willbarnes/ssw/packages/chianti/dbase/continuum/itoh.dat

In [ ]:
%%bash 
head -n 3 /Users/willbarnes/ssw/packages/chianti/dbase/continuum/heseq_2photon.dat

In [ ]:
%%bash 
head -n 5 /Users/willbarnes/ssw/packages/chianti/dbase/continuum/hseq_2photon.dat

## ChiantiPy View

In [ ]:
ch_io.gffRead()

In [ ]:
ch_io.gffintRead()

In [ ]:
ch_io.twophotonHeRead()

In [ ]:
ch_io.twophotonHRead()

In [ ]:
ch_io.vernerRead()['eth'].shape

In [ ]:
ch_io.vernerRead().keys()

In [ ]:
ch_io.klgfbRead()['klgfb'].shape

In [ ]:
ch_io.klgfbRead()['pe'].shape

In [ ]:
ch_io.itohRead().keys()

In [ ]:
ch_io.itohRead()['itohCoef'].shape

In [ ]:
with open('/Users/willbarnes/ssw/packages/chianti/dbase/h/h_1/h_1.elvlc','r') as f:
    lines = f.readlines()

In [ ]:
lines[77:]

In [ ]:
foo = lines[26:]

In [ ]:
foo

In [ ]:
foo[-1].strip('%').strip().strip('-1')

In [ ]:
foo[3].strip().strip('%')

In [ ]:
bar = '\n'.join([l.strip('%').strip() for l in foo if l.strip('%').strip().strip('-1')])

In [ ]:
bar

In [ ]:
print(bar)

In [ ]:
p = fiasco.io.Parser('sun_coronal_1992_feldman.abund')

In [ ]:
p.parse()

In [ ]:
p = fiasco.io.Parser('fe_15.wgfa')

In [ ]:
df = p.parse()

In [ ]:
print(df.meta['footer'])

In [ ]:
u.eV